In [4]:
import torch
import soundfile as sf
import pyaudio
import wave
from transformers import pipeline

# 1. Whisper 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
stt_pipeline = pipeline("automatic-speech-recognition", model="openai/whisper-small", device=device)

# 2. 음성 녹음 함수
def record_audio(filename, duration=5, sample_rate=16000):
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)

    print("녹음 시작...")
    frames = []

    for _ in range(0, int(sample_rate / 1024 * duration)):
        data = stream.read(1024)
        frames.append(data)

    print("녹음 종료.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()

# 3. 녹음한 파일을 텍스트로 변환
def audio_to_text(audio_path):
    # 오디오 파일 로드
    audio_input, _ = sf.read(audio_path)
    # STT 수행
    result = stt_pipeline(audio_input)["text"]
    print(f"인식된 텍스트: {result}")

# 4. 실행 예제
if __name__ == "__main__":
    audio_filename = "output.wav"
    record_audio(audio_filename, duration=5)  # 5초간 녹음
    audio_to_text(audio_filename)  # 텍스트 변환


녹음 시작...
녹음 종료.
인식된 텍스트: カーナーターは
